In [1]:
import ast
import datasets
import json
# import evaluate
import pandas as pd
import numpy as np
import torch

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, BertTokenizer, DataCollatorWithPadding, EvalPrediction
from sklearn.model_selection import train_test_split

c:\Users\mathi\Prosjekter\Barcelona\HLE\HLE-Project\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(torch.cuda.is_available())
device = "cuda:0" if torch.cuda.is_available() else "cpu"

True


In [3]:
df = pd.read_csv("BERT_input.csv")

In [4]:
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result
     

In [5]:
labels = [label for label in df.columns if label != 'text']
id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}
labels

['nyheter',
 'tema',
 'sport',
 'kjendis',
 'kultur',
 'meninger',
 'annonse',
 'bok',
 'magasinet',
 'okonomi']

In [6]:
model = AutoModelForSequenceClassification.from_pretrained(
    "NbAiLab/nb-bert-large",
    problem_type="multi_label_classification",
    num_labels=len(labels), 
    id2label=id2label, 
    label2id=label2id
)

Some weights of the model checkpoint at NbAiLab/nb-bert-large were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not init

In [7]:
tokenizer = BertTokenizer.from_pretrained("NbAiLab/nb-bert-large")

In [8]:
def preprocess_data(examples):
  # take a batch of texts
  text = examples["text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

In [9]:
dataset = datasets.Dataset.from_pandas(df).train_test_split(test_size=0.2)
dataset

DatasetDict({
    train: Dataset({
        features: ['nyheter', 'tema', 'sport', 'kjendis', 'kultur', 'meninger', 'annonse', 'bok', 'magasinet', 'okonomi', 'text'],
        num_rows: 13596
    })
    test: Dataset({
        features: ['nyheter', 'tema', 'sport', 'kjendis', 'kultur', 'meninger', 'annonse', 'bok', 'magasinet', 'okonomi', 'text'],
        num_rows: 3399
    })
})

In [10]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)
encoded_dataset.set_format("torch")
train = encoded_dataset["train"]
test = encoded_dataset["test"]

100%|██████████| 4/4 [00:04<00:00,  1.16s/ba]


In [11]:
train

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 13596
})

In [12]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [13]:
batch_size = 4
metric_name = "f1"

In [14]:
training_args = TrainingArguments(
    output_dir="E:\\Pytorch\\Saved_Models\\CLASSIFICATION_MODEL",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    metric_for_best_model=metric_name,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [15]:
example = encoded_dataset['train'][0]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [16]:
tokenizer.decode(example['input_ids'])

'[CLS] overskrift : avslorer rehab opphold matte innlegges. oppsummering : onsdag kom den første episoden av den ferske podkasten [UNK] sophie og fetisha [UNK] på nrk. personer : sophie elise, fetisha williams. nøkkelord : på, nrk, sophie, ferske, podkasten, sophie elise, fetisha williams, kjendis, podkast, influenser. spådd tema : tv, nrk, programmet, viaplay, viaplays, premier, dagbladet, tall, abonnenter, låt, melodi, prix, ukjent. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [17]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['kjendis']

In [18]:
trainer.train()

c:\Users\mathi\Prosjekter\Barcelona\HLE\HLE-Project\venv\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 13596
  Num Epochs = 5
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 16995
  Number of trainable parameters = 355097610
  3%|▎         | 500/16995 [03:26<1:51:45,  2.46it/s]

{'loss': 0.1093, 'learning_rate': 1.9411591644601354e-05, 'epoch': 0.15}


  6%|▌         | 1000/16995 [06:46<1:45:51,  2.52it/s]

{'loss': 0.0199, 'learning_rate': 1.882318328920271e-05, 'epoch': 0.29}


  9%|▉         | 1500/16995 [10:06<1:44:42,  2.47it/s]

...

100%|██████████| 16995/16995 [1:51:11<00:00,  2.60it/s]***** Running Evaluation *****
  Num examples = 3399
  Batch size = 4
                                                       
100%|██████████| 16995/16995 [1:51:55<00:00,  2.60it/s]

{'eval_loss': 0.007677115499973297, 'eval_f1': 0.9938217122683142, 'eval_roc_auc': 0.9965676179268412, 'eval_accuracy': 0.9938217122683142, 'eval_runtime': 44.1675, 'eval_samples_per_second': 76.957, 'eval_steps_per_second': 19.245, 'epoch': 5.0}


Saving model checkpoint to E:\Pytorch\Saved_Models\CLASSIFICATION_MODEL\checkpoint-16995
Configuration saved in E:\Pytorch\Saved_Models\CLASSIFICATION_MODEL\checkpoint-16995\config.json
Model weights saved in E:\Pytorch\Saved_Models\CLASSIFICATION_MODEL\checkpoint-16995\pytorch_model.bin
tokenizer config file saved in E:\Pytorch\Saved_Models\CLASSIFICATION_MODEL\checkpoint-16995\tokenizer_config.json
Special tokens file saved in E:\Pytorch\Saved_Models\CLASSIFICATION_MODEL\checkpoint-16995\special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from E:\Pytorch\Saved_Models\CLASSIFICATION_MODEL\checkpoint-13596 (score: 0.9938217122683142).
100%|██████████| 16995/16995 [1:52:14<00:00,  2.52it/s]

{'train_runtime': 6734.9401, 'train_samples_per_second': 10.094, 'train_steps_per_second': 2.523, 'train_loss': 0.008363515769919777, 'epoch': 5.0}


TrainOutput(global_step=16995, training_loss=0.008363515769919777, metrics={'train_runtime': 6734.9401, 'train_samples_per_second': 10.094, 'train_steps_per_second': 2.523, 'train_loss': 0.008363515769919777, 'epoch': 5.0})

In [19]:
trainer.save_model("E:\\Pytorch\\Saved_Models\\CLASSIFICATION_MODEL\\FINAL")

Saving model checkpoint to E:\Pytorch\Saved_Models\CLASSIFICATION_MODEL\FINAL
Configuration saved in E:\Pytorch\Saved_Models\CLASSIFICATION_MODEL\FINAL\config.json
Model weights saved in E:\Pytorch\Saved_Models\CLASSIFICATION_MODEL\FINAL\pytorch_model.bin
tokenizer config file saved in E:\Pytorch\Saved_Models\CLASSIFICATION_MODEL\FINAL\tokenizer_config.json
Special tokens file saved in E:\Pytorch\Saved_Models\CLASSIFICATION_MODEL\FINAL\special_tokens_map.json
